In [1]:
# -*- coding: utf-8 -*-
import sys
sys.path.append('/home/jun/PersonReID')
import torch
from torch import nn, optim
from torch.nn import init
from torchvision import datasets, transforms 
from torchinfo import summary
import matplotlib
import time 
import os
import tqdm.auto as tqdm
from model import * 
from utils.random_erasing import RandomErasing
from utils.dgfolder import DGFolder
import yaml
from shutil import copyfile
from utils.circle_loss import CircleLoss, convert_label_to_similarity
from utils.instance_loss import InstanceLoss
from online_triplet_loss.losses import *
import timm
from utils.ODFA import ODFA
proxy = 'http://10.0.0.107:3128'
os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [ ]:
class PSPViT(nn.Module):
    def __init__(self, class_num=751, droprate=0.5, stride=2, circle=False, ibn=False, linear_num=256):
        super().__init__()
        model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=class_num)
        self.patch_embed = model.patch_embed
        self.pos_drop = model.pos_drop
        self.cls_token = model.cls_token
        self.pos_embed = model.pos_embed
        self.patch_drop = model.patch_drop
        self.norm_pre = model.norm_pre
        self.blocks = model.blocks
        self.norm = model.norm
        self.fc_norm = model.fc_norm
        self.avgpool = nn.AdaptiveAvgPool2d((1,768))
        self.dropout = nn.Dropout(p=0.5)
        self.classifier = ClassBlock(768, class_num, droprate, linear=linear_num, return_f = circle)
    def forward(self, x):
        x = self.patch_embed(x)
        cls_token = self.cls_token.expand(x.shape[0], -1, -1) 
        x = torch.cat((cls_token, x), dim=1)
        x = self.pos_drop(x + self.pos_embed)
        x = self.patch_drop(x)
        x = self.norm_pre(x)
        x = self.blocks(x) # torch.Size([32, 197, 768])
        x = self.norm(x) # torch.Size([32, 197, 768])
        x = self.fc_norm(x) # torch.Size([32, 197, 768]) 
        x = self.avgpool(x) # torch.Size([32, 197, 768]) 
        x = self.dropout(x)
        x = x.view(x.shape[0], x.shape[2])
        x = self.classifier(x)
        return x

In [ ]:
netCopilot

In [11]:
model_train = models.resnet18(pretrained=True).to('cpu')
#model_train.load_state_dict(torch.load('../model/LASwin_all_trick_duke/net_last.pth'))
#model_train.classifier.classifier = nn.Sequential() 
img = torch.rand((32,3,224,224)).to('cpu')
#outputs = model_train(img)

/home/jun/miniconda3/envs/py311/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jun/miniconda3/envs/py311/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
#model_train = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)

Using cache found in /home/jun/.cache/torch/hub/pytorch_vision_v0.10.0


In [12]:
model_train

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
model_train

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
summary(model=model_train, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Conv2d: 1, BatchNorm2d: 1, ReLU: 1, MaxPool2d: 1, Sequential: 1, BasicBlock: 2, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, BasicBlock: 2, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Sequential: 1, BasicBlock: 2, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, Sequential: 3, Conv2d: 4, BatchNorm2d: 4, ReLU: 3, BasicBlock: 2, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Sequential: 1, BasicBlock: 2, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, Sequential: 3, Conv2d: 4, BatchNorm2d: 4, ReLU: 3, BasicBlock: 2, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, Sequential: 3, Conv2d: 4, BatchNorm2d: 4]

In [7]:
class LATransformerTest(nn.Module):
    def __init__(self, lmbd ):
        super(LATransformerTest, self).__init__()
        model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=751)  
        self.class_num = 751
        self.part = 14 # We cut the pool5 to sqrt(N) parts
        self.num_blocks = 12
        self.model = model
        self.model.head.requires_grad_ = False 
        self.cls_token = self.model.cls_token
        self.pos_embed = self.model.pos_embed
        self.avgpool = nn.AdaptiveAvgPool2d((self.part,768))
        self.dropout = nn.Dropout(p=0.5)
        self.lmbd = lmbd
    
    def forward(self,x):
        # Divide input image into patch embeddings and add position embeddings
        x = self.model.patch_embed(x)
        cls_token = self.cls_token.expand(x.shape[0], -1, -1) 
        x = torch.cat((cls_token, x), dim=1)
        x = self.model.pos_drop(x + self.pos_embed)
         
        # Feed forward through transformer blocks
        for i in range(self.num_blocks):
            x = self.model.blocks[i](x)
        x = self.model.norm(x)
         
        # extract the cls token
        cls_token_out = x[:, 0].unsqueeze(1)
         
        # Average pool
        x = self.avgpool(x[:, 1:])
        x = x.permute(0,2,1)
        y = x.view(x.size(0),x.size(1),x.size(2))
          
        return y

In [8]:
model_test = LATransformerTest(lmbd=8)
model_test

LATransformerTest(
  (model): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='n

In [9]:
vit = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=751)
summary(model=vit, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)    [32, 3, 224, 224]    [32, 751]            152,064              True
├─PatchEmbed (patch_embed)               [32, 3, 224, 224]    [32, 196, 768]       --                   True
│    └─Conv2d (proj)                     [32, 3, 224, 224]    [32, 768, 14, 14]    590,592              True
│    └─Identity (norm)                   [32, 196, 768]       [32, 196, 768]       --                   --
├─Dropout (pos_drop)                     [32, 197, 768]       [32, 197, 768]       --                   --
├─Identity (patch_drop)                  [32, 197, 768]       [32, 197, 768]       --                   --
├─Identity (norm_pre)                    [32, 197, 768]       [32, 197, 768]       --                   --
├─Sequential (blocks)                    [32, 197, 768]       [32, 197, 768]       --                   True
│    └─Block (0)      

In [10]:
vit.blocks[-1].norm1

LayerNorm((768,), eps=1e-06, elementwise_affine=True)

In [11]:
class ViTReID(nn.Module):
    def __init__(self, class_num=751, droprate=0.5, stride=2, circle=False, ibn=False, linear_num=512):
        super().__init__()
        model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=class_num)
        self.patch_embed = model.patch_embed
        self.pos_drop = model.pos_drop
        self.cls_token = model.cls_token
        self.pos_embed = model.pos_embed
        self.patch_drop = model.patch_drop
        self.norm_pre = model.norm_pre
        self.blocks = model.blocks
        self.norm = model.norm
        self.fc_norm = model.fc_norm
        self.avgpool = nn.AdaptiveAvgPool2d((1,768))
        self.dropout = nn.Dropout(p=0.5)
        self.classifier = ClassBlock(768, class_num, droprate, linear=linear_num, return_f = circle)
    def forward(self, x):
        x = self.patch_embed(x)
        cls_token = self.cls_token.expand(x.shape[0], -1, -1) 
        x = torch.cat((cls_token, x), dim=1)
        print(self.pos_embed.shape)
        x = self.pos_drop(x + self.pos_embed)
        x = self.patch_drop(x)
        x = self.norm_pre(x)
        x = self.blocks(x)
        x = self.norm(x)
        x = self.fc_norm(x)
        x = self.avgpool(x)
        x = self.dropout(x)
        x = x.view(x.shape[0], x.shape[2])
        x = self.classifier(x)
        return x

In [12]:
model = ViTReID()

In [13]:
summary(model=model, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

torch.Size([1, 197, 768])


Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ViTReID (ViTReID)                        [32, 3, 224, 224]    [32, 751]            152,064              True
├─PatchEmbed (patch_embed)               [32, 3, 224, 224]    [32, 196, 768]       --                   True
│    └─Conv2d (proj)                     [32, 3, 224, 224]    [32, 768, 14, 14]    590,592              True
│    └─Identity (norm)                   [32, 196, 768]       [32, 196, 768]       --                   --
├─Dropout (pos_drop)                     [32, 197, 768]       [32, 197, 768]       --                   --
├─Identity (patch_drop)                  [32, 197, 768]       [32, 197, 768]       --                   --
├─Identity (norm_pre)                    [32, 197, 768]       [32, 197, 768]       --                   --
├─Sequential (blocks)                    [32, 197, 768]       [32, 197, 768]       --                   True
│    └─Block (0)      

In [14]:
optim_name = optim.SGD #apex.optimizers.FusedSGD
ignored_params = list(map(id, model.classifier.parameters() ))
base_params = filter(lambda p: id(p) not in ignored_params, model.parameters())
classifier_params = model.classifier.parameters()
optimizer_ft = optim_name([
        {'params': base_params, 'lr': 0.1*0.01},
        {'params': classifier_params, 'lr': 0.01}
    ], weight_decay=5e-4, momentum=0.9, nesterov=True)

In [15]:
# Decay LR by a factor of 0.1 every 40 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=9*2//3, gamma=0.1)
criterion = nn.CrossEntropyLoss()

In [16]:
def train_loop(model,
               data_loader,
               loss_fn,
               optimizer,
               device):
    accuracy_fn = accuracy_fn.to(device)
    train_acc , train_loss = 0, 0
    criterion_circle = CircleLoss(m=0.25, gamma=32) # gamma = 64 may lead to a better result.
    for batch_num, (inputs,labels) in tqdm(enumerate(data_loader)):
        model.train()
        inputs, labels = inputs.to(device), labels.to(device)
        now_batch_size,c,h,w = inputs.shape
        logits, ff = model(inputs)
        fnorm = torch.norm(ff, p=2, dim=1, keepdim=True)
        ff = ff.div(fnorm.expand_as(ff))
        loss = criterion(logits, labels) 
        loss += criterion_circle(*convert_label_to_similarity( ff, labels))/now_batch_size
        train_loss += loss
        train_acc += accuracy_fn(y_preds, label)
        optimizer.zero_grad()
        loss = criterion(logits, labels) 
        loss.backward()
        optimizer.step()
        
    train_acc /= len(data_loader)
    train_loss /= len(data_loader)
    print(f"Train acc: {train_acc*100:.2f} | Train loss: {train_loss:.4f}")
    
def test_loop(model,
               data_loader,
               loss_fn,
               optimizer,
               device):
    accuracy_fn = accuracy_fn.to(device)
    test_acc , test_loss = 0, 0
    with torch.inference_mode():
        for batch_num, (image,label) in tqdm(enumerate(data_loader)):
            model.eval
            image, label = image.to(device), label.to(device)
            y_logits = model(image)
            y_preds = y_logits 
            loss = loss_fn(y_logits, label)
            test_loss += loss
            test_acc += accuracy_fn(y_preds, label)
        test_acc /= len(data_loader)
        test_loss /= len(data_loader)
    print(f"Test acc: {test_acc*100:.2f} | Test loss: {test_loss:.4f}")

In [17]:
class ViTReID(nn.Module):
    def __init__(self, class_num=751, droprate=0.5, stride=2, circle=False, ibn=False, linear_num=512):
        super().__init__()
        model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=class_num)
        self.patch_embed = model.patch_embed
        self.pos_drop = model.pos_drop
        self.patch_drop = model.patch_drop
        self.norm_pre = model.norm_pre
        self.blocks = model.blocks
        self.norm = model.norm
        self.fc_norm = model.fc_norm
        self.avgpool = nn.AdaptiveAvgPool2d((1,768))
        self.dropout = nn.Dropout(p=0.5)
        self.classifier = ClassBlock(768, class_num, droprate, linear=linear_num, return_f = circle)
    def forward(self, x):
        x = self.patch_embed(x)
        x = self.pos_drop(x)
        x = self.patch_drop(x)
        x = self.norm_pre(x)
        x = self.blocks(x)
        x = self.norm(x)
        x = self.fc_norm(x)
        x = self.avgpool(x)
        x = self.dropout(x)
        x = x.view(x.shape[0], x.shape[2])
        x = self.classifier(x)
        return x

In [18]:
model_swin = model_ft = timm.create_model('swin_base_patch4_window7_224', pretrained=True, drop_path_rate = 0.2)

/home/jun/miniconda3/envs/py311/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1699449201450/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [19]:
summary(model=model_swin, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                            Input Shape          Output Shape         Param #              Trainable
SwinTransformer (SwinTransformer)                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─PatchEmbed (patch_embed)                         [32, 3, 224, 224]    [32, 56, 56, 128]    --                   True
│    └─Conv2d (proj)                               [32, 3, 224, 224]    [32, 128, 56, 56]    6,272                True
│    └─LayerNorm (norm)                            [32, 56, 56, 128]    [32, 56, 56, 128]    256                  True
├─Sequential (layers)                              [32, 56, 56, 128]    [32, 7, 7, 1024]     --                   True
│    └─SwinTransformerStage (0)                    [32, 56, 56, 128]    [32, 56, 56, 128]    --                   True
│    │    └─Identity (downsample)                  [32, 56, 56, 128]    [32, 56, 56, 128]    --                   --
│    │    └─Sequential (blocks)              

In [20]:
class LATransformer(nn.Module):
    def __init__(self, lmbd ):
        super(LATransformer, self).__init__()
        model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=751) 
        self.class_num = 751
        self.part = 14 # We cut the pool5 to sqrt(N) parts
        self.num_blocks = 12
        self.model = model
        self.model.head.requires_grad_ = False 
        self.cls_token = self.model.cls_token
        self.pos_embed = self.model.pos_embed
        self.avgpool = nn.AdaptiveAvgPool2d((self.part,768))
        self.dropout = nn.Dropout(p=0.5)
        self.lmbd = lmbd
        for i in range(self.part):
            name = 'classifier'+str(i)
            setattr(self, name, ClassBlock(768, self.class_num, droprate=0.5, relu=False, bnorm=True, num_bottleneck=256))

    def forward(self,x):
        # Divide input image into patch embeddings and add position embeddings
        x = self.model.patch_embed(x)
        cls_token = self.cls_token.expand(x.shape[0], -1, -1) 
        x = torch.cat((cls_token, x), dim=1)
        x = self.model.pos_drop(x + self.pos_embed)
        
        # Feed forward through transformer blocks
        for i in range(self.num_blocks):
            x = self.model.blocks[i](x)
        x = self.model.norm(x)
        
        # Extract the cls token
        cls_token_out = x[:, 0].unsqueeze(1)
        
        # Average pool
        x = self.avgpool(x[:, 1:])
        # Add global cls token to each local token 
        for i in range(self.part):
            out = torch.mul(x[:, i, :], self.lmbd)
            x[:,i,:] = torch.div(torch.add(cls_token_out.squeeze(),out), 1+self.lmbd)
        x = x.permute(0,2,1)
        # Locally aware network
        part = {}
        predict = {}
        
        for i in range(self.part):
            #part[i] = x[:,i,:]
            part[i] = x[:,:,i].view(x.size(0), x.size(1))
            name = 'classifier'+str(i)
            c = getattr(self,name)
            predict[i] = c(part[i])
        
        return predict

In [21]:
lmbd = 8
la_tran =  LATransformer(lmbd)
summary(model=la_tran, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

TypeError: ClassBlock.__init__() got an unexpected keyword argument 'num_bottleneck'

In [ ]:
model = build_transformer_local(num_class, camera_num, view_num, cfg, __factory_T_type, rearrange=cfg.MODEL.RE_ARRANGE)